### Задание

Напишем нейронную сеть, которая может стать составной частью системы бота для игры в "Крестики-нолики". Используя подготовленную базу изображений, создадим и обучим нейронную сеть, распознающую две категории изображений: крестики и нолики. Добьемся точности распознавания более 95% (accuracy)


In [1]:
# Подключение класса для создания нейронной сети прямого распространения
from tensorflow.keras.models import Sequential
# Подключение класса для создания полносвязного слоя
from tensorflow.keras.layers import Dense
# Подключение оптимизатора
from tensorflow.keras.optimizers import Adam
# Подключение утилит для to_categorical
from tensorflow.keras import utils
# Подключение библиотеки для загрузки изображений
from tensorflow.keras.preprocessing import image
# Подключение библиотеки для работы с массивами
import numpy as np
# Подключение модуля для работы с файлами
import os
# Подключение библиотек для отрисовки изображений
import matplotlib.pyplot as plt
from PIL import Image
# Вывод изображения в ноутбуке, а не в консоли или файле
%matplotlib inline

In [2]:
# Загрузка датасета из облака
import gdown
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l3/hw_pro.zip', None, quiet=True)

'hw_pro.zip'

In [3]:
# Распаковываем архив hw_light.zip в папку hw_light
!unzip -q hw_pro.zip

In [4]:
# Путь к директории с базой
base_dir = '/content/hw_pro'
# Создание пустого списка для загрузки изображений обучающей выборки
x_train = []
# Создание списка для меток классов
y_train = []
# Задание высоты и ширины загружаемых изображений
img_height = 20
img_width = 20
# Перебор папок в директории базы
for patch in os.listdir(base_dir):
    # Перебор файлов в папках
    for img in os.listdir(base_dir + '/' + patch):
        # Добавление в список изображений текущей картинки
        x_train.append(image.img_to_array(image.load_img(base_dir + '/' + patch + '/' + img,
                                                         target_size=(img_height, img_width),
                                                         color_mode='grayscale')))
        # Добавление в массив меток, соответствующих классам
        if patch == '0':
            y_train.append(0)
        else:
            y_train.append(1)
# Преобразование в numpy-массив загруженных изображений и меток классов
x_train = np.array(x_train)
y_train = np.array(y_train)
# Вывод размерностей
print('Размер массива x_train', x_train.shape)
print('Размер массива y_train', y_train.shape)

Размер массива x_train (102, 20, 20, 1)
Размер массива y_train (102,)


In [5]:
# Изменение формы входных картинок с 28х28 на 784
# первая ось остается без изменения, остальные складываются в вектор
x_train = x_train.reshape(x_train.shape[0], -1)

# Проверка результата
print(f'Форма обучающих данных:  -> {x_train.shape}')


Форма обучающих данных:  -> (102, 400)


In [6]:
y_train.shape

(102,)

In [7]:
y_train[1]

1

In [8]:
x_train = x_train.astype('float32') / 255.
CLASS_COUNT = 10
y_train = utils.to_categorical(y_train, CLASS_COUNT)
print(y_train.shape)

(102, 10)


In [9]:
y_train[0]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [10]:
#создадим нашу модель
model = Sequential()
model.add(Dense(800, input_dim=400, activation='relu', name='Dense_800'))
model.add(Dense(400, activation='relu', name='Dense_400'))
model.add(Dense(CLASS_COUNT, activation='softmax', name='Dense_10'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
#проверим слои нашей модели
model.layers

[<Dense name=Dense_800, built=True>,
 <Dense name=Dense_400, built=True>,
 <Dense name=Dense_10, built=True>]

In [12]:
#скомпилируем нашу модель и выведем информацию о слоях
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Dense_800 (Dense)                    │ (None, 800)                 │         320,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Dense_400 (Dense)                    │ (None, 400)                 │         320,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Dense_10 (Dense)                     │ (None, 10)                  │           4,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 645,210 (2.46 MB)

 Trainable params: 645,210 (2.46 MB)

 Non-trainable params: 0 (0.00 B)

None


In [13]:
#подключим GoogleDisk
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [14]:
#обучим модель на 25 эпохах
model.fit(x_train,
          y_train,
          batch_size=128,
          epochs=25,
          verbose=1)

Epoch 1/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 2.1601
Epoch 2/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6275 - loss: 0.6372
Epoch 3/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.4902 - loss: 1.2759
Epoch 4/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5098 - loss: 0.8469
Epoch 5/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5098 - loss: 0.7714
Epoch 6/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5784 - loss: 0.5629
Epoch 7/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.4902 - loss: 0.7313
Epoch 8/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.9020 - loss: 0.4831
Epoch 9/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5098 - loss: 0.5839
Epoch 10/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5392 - loss: 0.5389
Epoch 11/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.9706 - loss: 0.3995
Epoch 12/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.6275 - loss: 0.4939

In [15]:
#Сохраним нашу модель
from keras.models import load_model

model.save('model.keras')

model = load_model('model.keras')